# Demonstration of WellResponse Class

Notebook to demonstrate using the Well Class to 
estimate drawdown or stream depletion using the 
solutions available in the python module.

In [1]:
import sys
sys.path.insert(1, '../../')
import pycap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Example Depletion
The Well class will estimate depletions for streams.  The stream distances and
streambed conductances, if needed, are passed through dictionaries keyed by 
a stream name or ID.  In this notebook a table of values will be made manually from 
Table 2 from Reeves, H.W., Hamilton, D.A., Seelbach, P.W., and Asher, A.J., 2009, Ground-water-withdrawal component of the Michigan water-withdrawal screening tool: U.S. Geological Survey Scientific Investigations Report 2009–5003, 36 p.
[https://pubs.usgs.gov/sir/2009/5003/]

In [2]:
stream_table = pd.DataFrame(({'id': 8, 'distance': 14802},
                            {'id': 9, 'distance': 12609.2},
                            {'id': 11, 'distance': 15750.5},
                            {'id': 27, 'distance': 22567.6},
                            {'id': 9741, 'distance': 27565.2},
                            {'id': 10532, 'distance': 33059.5},
                            {'id': 11967, 'distance': 14846.3},
                            {'id': 12515, 'distance': 17042.5},
                            {'id': 12573, 'distance': 11959.5},
                            {'id': 12941, 'distance': 19070.8},
                            {'id': 13925, 'distance': 10028.9}))



In [3]:
stream_table.head()

,id,distance
0,8,14802.0
1,9,12609.2
2,11,15750.5
3,27,22567.6
4,9741,27565.2


# Compute apportionment

The Michigan screening tool uses inverse distance weighting apportionment.  Other
apportionment approaches that may be used as a simple way to extend the analytical
solution are discussed by Zipper and others (2019).  [https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2018WR024403]

In [4]:
sum_dist = np.sum(stream_table['distance'].values)
stream_table['apportionment'] = stream_table['distance']/sum_dist

In [5]:
stream_table.head()

,id,distance,apportionment
0,8,14802.0,0.074269
1,9,12609.2,0.063267
2,11,15750.5,0.079028
3,27,22567.6,0.113233
4,9741,27565.2,0.138309


In [6]:
np.sum(stream_table['apportionment'])

np.float64(1.0000000000000002)

make dictionaries from the distances and apportionment values

In [7]:
distances = dict(zip(stream_table.id.values, stream_table.distance.values))

In [8]:
apportion = dict(zip(stream_table.id.values, stream_table.apportionment.values))

# set aquifer properties and streambed conductances

In [9]:
stream_table = pd.DataFrame(({'id': 8, 'distance': 14802},
                            {'id': 9, 'distance': 12609.2},
                            {'id': 11, 'distance': 15750.5},
                            {'id': 27, 'distance': 22567.6},
                            {'id': 9741, 'distance': 27565.2},
                            {'id': 10532, 'distance': 33059.5},
                            {'id': 11967, 'distance': 14846.3},
                            {'id': 12515, 'distance': 17042.5},
                            {'id': 12573, 'distance': 11959.5},
                            {'id': 12941, 'distance': 19070.8},
                            {'id': 13925, 'distance': 10028.9}))

In [12]:
T= 7211.  # ft^2/day
S= 0.01
Q = 70  # 70 gpm in cubic feet per day
well_name = 'demo'
pumpdays = int(5. * 365)

## call a utility to convert Q to a series with appropriate formatting

the utility also converts to CFD from GPM

In [13]:
Q = pycap.Q2ts(pumpdays, 5, Q)

In [14]:
Q

1       13475.935829
2       13475.935829
3       13475.935829
4       13475.935829
5       13475.935829
            ...     
1821    13475.935829
1822    13475.935829
1823    13475.935829
1824    13475.935829
1825    13475.935829
Length: 1825, dtype: float64

In [15]:
stream_table['conductance'] = np.array([200.051, 414.302, 193.671, 180.88, 324.59300000000, 680.98300000000, 569.48400000000,
                               194.03500000000, 396.06600000000, 118.20700000000, 73.68700000000])/80.

In [16]:
stream_table.head()

,id,distance,conductance
0,8,14802.0,2.500637
1,9,12609.2,5.178775
2,11,15750.5,2.420888
3,27,22567.6,2.261000
4,9741,27565.2,4.057412


In [17]:
cond = dict(zip(stream_table.id.values, stream_table.conductance.values))

In [18]:
test_well = pycap.Well(T=T,
                 S=S,
                 Q=Q,
                 depletion_years=5,
                 depl_method='hunt99',
                 streambed_conductance=cond,
                 stream_dist=distances,
                 stream_apportionment=apportion)

In [19]:
deltaQ = pycap._calc_deltaQ(Q)
deltaQ

1    13475.935829
dtype: float64

In [20]:
test_well.depletion

c:\Home-Local\pycap\source\examples\../..\pycap\solutions.py:558: RuntimeWarning: divide by zero encountered in divide
  a = np.sqrt(S * dist**2 / (4.0 * T * time))


{np.int64(8): array([0.00000000e+00, 1.25358594e-38, 1.07003823e-21, ...,
        7.99669021e-03, 7.99761056e-03, 7.99853019e-03], shape=(1825,)),
 np.int64(9): array([0.00000000e+00, 3.31995363e-29, 8.23422835e-17, ...,
        7.54212679e-03, 7.54274389e-03, 7.54336049e-03], shape=(1825,)),
 np.int64(11): array([0.00000000e+00, 4.95920323e-43, 6.45276083e-24, ...,
        8.31128705e-03, 8.31231014e-03, 8.31333245e-03], shape=(1825,)),
 np.int64(27): array([0.00000000e+00, 1.53027500e-82, 9.33445162e-44, ...,
        1.01633853e-02, 1.01652072e-02, 1.01670278e-02], shape=(1825,)),
 np.int64(9741): array([0.00000000e+000, 4.24127261e-120, 1.87831469e-062, ...,
        1.17525717e-002, 1.17549405e-002, 1.17573076e-002], shape=(1825,)),
 np.int64(10532): array([0.00000000e+000, 5.17752884e-170, 2.70879966e-087, ...,
        1.28848932e-002, 1.28879400e-002, 1.28909846e-002], shape=(1825,)),
 np.int64(11967): array([0.00000000e+00, 2.19107381e-38, 2.28636967e-21, ...,
        8.62391038e

In [21]:
stream_depl = pd.DataFrame(test_well.depletion)

## need to convert to GPM to compare results to Table 2

In [22]:
stream_depl = stream_depl * 448.83116885

In [23]:
stream_depl

,8,9,11,27,9741,10532,11967,12515,12573,12941,13925
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,5.626484e-36,1.490099e-26,2.225845e-40,6.868351e-80,1.903615e-117,2.323836e-167,9.834222e-36,8.635939e-47,3.790560e-24,4.440575e-58,2.111829e-18
2,4.802665e-19,3.695778e-14,2.896200e-21,4.189593e-41,8.430462e-60,1.215794e-84,1.026194e-18,1.744600e-24,5.893776e-13,2.958061e-30,2.129281e-10
3,2.689067e-13,6.263964e-10,8.679513e-15,4.562716e-28,1.775652e-40,5.800727e-57,6.048305e-13,6.065673e-17,3.963026e-09,7.143521e-21,1.252784e-07
4,2.251477e-10,9.032354e-08,1.683464e-11,1.696089e-21,9.171113e-31,4.492188e-43,5.141865e-10,4.012408e-13,3.596457e-07,3.952945e-16,3.388312e-06
...,...,...,...,...,...,...,...,...,...,...,...
1820,3.588337e+00,3.384587e+00,3.729445e+00,4.560007e+00,5.272792e+00,5.780404e+00,3.869969e+00,3.927259e+00,3.242961e+00,3.892709e+00,2.234948e+00
1821,3.588750e+00,3.384864e+00,3.729905e+00,4.560826e+00,5.273857e+00,5.781773e+00,3.870324e+00,3.927781e+00,3.243216e+00,3.893380e+00,2.235247e+00
1822,3.589164e+00,3.385142e+00,3.730365e+00,4.561644e+00,5.274920e+00,5.783142e+00,3.870680e+00,3.928302e+00,3.243470e+00,3.894050e+00,2.235546e+00
1823,3.589577e+00,3.385419e+00,3.730824e+00,4.562462e+00,5.275984e+00,5.784509e+00,3.871035e+00,3.928823e+00,3.243725e+00,3.894720e+00,2.235844e+00
